#### Using colab as the basic hardware platform, So we need to change the colab tensorflow version to 2.1.0 and mount the Google Drive.
# ===============================================
#### 使用colab作为基础硬件平台，因此我们需要将colab tensorflow版本更改为2.1.0并安装谷歌网盘驱动器。

In [ ]:
import sys
sys.path[0] = '/tensorflow-2.1.0/python3.6'
from google.colab import drive
drive.mount('/drive')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, losses, optimizers, datasets
from tensorflow.keras.utils import to_categorical

In [ ]:
tf.__version__

#### Read cifar100 data from keras.datasets 
# ===============================================
#### 从keras.datasets 中读取cifar100 数据

In [ ]:
(x, y), (x_test, y_test) = datasets.cifar100.load_data()
print(x.shape,y.shape,x_test.shape,y_test.shape)
print(x.dtype,y.dtype)

#### Preprocess the data type and values. Normalize x and embedding y to one_hot format and reshape the y dimension to tf accessable dimension.
# ===============================================
#### 预处理数据类型和值。将x标准化并将y转换到one_hot格式，并将y维度重塑为tf可访问维度。

In [ ]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(to_categorical(tf.squeeze(tf.cast(y, dtype=tf.int32), axis=1), num_classes=100), dtype=tf.int32)
    return x,y

In [ ]:
class VGG16(keras.Model):
    def __init__(self):
        super(VGG16, self).__init__()
        self.VGG16 = Sequential([
            layers.Conv2D(64, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Conv2D(64, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(rate=0.5),
            layers.MaxPool2D(pool_size=[2,2], strides=2, padding="same"),

            layers.Conv2D(128, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(128, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=[2,2], strides=2, padding="same"),

            layers.Conv2D(256, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(256, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(256, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=[2,2], strides=2, padding="same"),

            layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=[2,2], strides=2, padding="same"),
            
            layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
            layers.BatchNormalization(),
            layers.MaxPool2D(pool_size=[2,2], strides=2, padding="same"),     

            layers.Flatten(),
            layers.Dense(4096, activation=tf.nn.relu),
            layers.Dropout(rate=0.5),
            layers.Dense(4096, activation=tf.nn.relu),
            layers.Dropout(rate=0.5),
            layers.Dense(100, activation=tf.nn.softmax)
        ])
        
    def call(self, inputs, training=None):
        x = inputs
        prediction = self.VGG16(x)
        return prediction

In [ ]:
def main(x, y, x_test, y_test):
    epochs = 1000
    model = VGG16()
    model.build(input_shape=(None, 32, 32, 3))
    model.summary()
    save_best = keras.callbacks.ModelCheckpoint('/drive/My Drive/Github/CNN/VGG16_best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, min_delta=0, patience=100, mode='auto')
    callbacks_list = [early_stop, save_best]
    model.compile(optimizer=optimizers.Adam(),
                 loss=losses.categorical_crossentropy,
                 metrics=['accuracy'])
    x, y = preprocess(x, y)
    x_test, y_test = preprocess(x_test, y_test)
    history = model.fit(x=x, y=y, epochs=epochs, batch_size=512, validation_data=(x_test, y_test), verbose=1, callbacks=callbacks_list)
    return history

In [ ]:
history = main(x, y, x_test, y_test)